In [5]:
import pandas as pd
import requests
from PIL import Image
from io import BytesIO

In [17]:


# Function to get game data from Speedrun API
def get_game_cover(game_id):
    url = f"https://www.speedrun.com/api/v1/games/{game_id}"
    
    # Send GET request to fetch game data
    response = requests.get(url)
    
    if response.status_code == 200:
        # Parse the response as JSON
        game_data = response.json()
        
        
        print(game_data['data']['assets'])
        # Extract the cover image URL
        cover_url = game_data['data']['assets']['cover-small']['uri']
        icon_url = game_data['data']['assets']['icon']['uri']
        
        # Print the cover URL
        # print(f"Cover URL: {cover_url}")
        
        # Download the cover image
        cover_response = requests.get(cover_url)
        icon_response = requests.get(icon_url)
        
        if cover_response.status_code == 200:
            # Open the image from the response content
            image = Image.open(BytesIO(cover_response.content))
            # Save as webp to disk
            image.save(f"output/images/{id}_cover.webp", "WEBP")
            #image.show()  # Display the image
            #return image
        else:
            print("Failed to download the image.")
            #return None
    
        if icon_response.status_code == 200:
            print('DOWNLOADING ICON')
            image = Image.open(BytesIO(icon_response.content))
            image.save(f"output/images/{id}_icon.webp", "WEBP")
            return image
        else:
            print("Failed to download the icon.")
            # return None
    else:
        print("Failed to retrieve game data.")
        return None


In [18]:
# Load the data
data = pd.read_feather('data/temp/data.feather')

# Data games
data_games = pd.read_csv('data/v1/games-data.csv')


In [19]:
# Find the games ("name") with the most submissions, and keep the top 100, along with their release date and platform
top_games = data[["name", "name_category"]].value_counts()

# Convert the index to columns
top_games = top_games.reset_index()

# For each game, find the first release date and platform
top_games_data = data[data["name"].isin(top_games["name"])].groupby(["name", "name_category"]).agg(
    {"releaseDate": "first", "platform_name": "first"}
)


# Add count of submissions for each game and category in the top games
top_games_data = top_games_data.join(data[["name", "name_category"]].value_counts(), on=["name", "name_category"])



# Add sum of all submissions for each game in the top games
top_games_data["sum_count"] = top_games_data["count"].groupby("name").transform("sum")



# Only keep the top 3 count of submissions for each game...
# First sort by count of submissions (descending) for each game
top_games_data = top_games_data.sort_values("count", ascending=False)


# Sort by count of submissions (descending)
top_games_data = top_games_data.sort_values(["sum_count", "count"], ascending=False)

# Remove any rown that have NaN or None
top_games_data = top_games_data.dropna()

# Reset the index
top_games_data = top_games_data.reset_index()


top_50_games_data_names = top_games_data.drop_duplicates(subset=["name"]).head(50)

In [20]:

top_50_games_data_names

,name,name_category,releaseDate,platform_name,count,sum_count
0,Super Mario 64,16 Star,1996-06-23,Nintendo 64,17485,34823
5,Super Mario Odyssey,Any%,2017-10-27,Switch,14525,17811
11,Minecraft: Java Edition,Any% Glitchless,2011-11-18,PC,15784,17185
21,Mario Kart 8 Deluxe,48 Tracks,2017-04-28,Switch,3374,16151
31,Seterra (Old Version),Europe: Countries,1997-01-01,Web,3897,14219
52,Celeste,Any%,2018-01-25,PC,11404,14196
63,Portal,Glitchless,2007-10-09,PC,4564,11210
67,Super Metroid,Any%,1994-03-19,Super Nintendo,5406,10684
78,Getting Over It With Bennett Foddy,Glitchless,2017-10-06,PC,9861,10415
80,Super Mario Sunshine,Any%,2002-07-19,GameCube,7107,10249


In [21]:
# Do a inner join with the data_games to get the game_id
top_50_games_data = pd.merge(top_50_games_data_names, data_games, on='name')

In [22]:
top_50_games_data

,name,name_category,releaseDate_x,platform_name,count,sum_count,ID,URL,releaseDate_y,createdDate,numCategories,numLevels
0,Super Mario 64,16 Star,1996-06-23,Nintendo 64,17485,34823,o1y9wo6q,sm64,1996-06-23,NaN,6,15
1,Super Mario Odyssey,Any%,2017-10-27,Switch,14525,17811,76r55vd8,smo,2017-10-27,2017-09-27T00:35:19Z,6,0
2,Minecraft: Java Edition,Any% Glitchless,2011-11-18,PC,15784,17185,j1npme6p,mc,2011-11-18,2015-01-29T23:41:21Z,15,0
3,Mario Kart 8 Deluxe,48 Tracks,2017-04-28,Switch,3374,16151,46w0n91r,mk8dx,2017-04-28,2017-04-30T00:26:59Z,10,0
4,Seterra (Old Version),Europe: Countries,1997-01-01,Web,3897,14219,k6q4rqzd,seterraold,1997-01-01,2018-09-27T08:29:37Z,29,903
5,Celeste,Any%,2018-01-25,PC,11404,14196,o1y9j9v6,celeste,2018-01-25,2018-01-15T20:23:45Z,15,9
6,Portal,Glitchless,2007-10-09,PC,4564,11210,4pd0n31e,portal,2007-10-09,NaN,7,24
7,Super Metroid,Any%,1994-03-19,Super Nintendo,5406,10684,m1zoemd0,supermetroid,1994-03-19,2016-11-25T20:23:44Z,11,0
8,Getting Over It With Bennett Foddy,Glitchless,2017-10-06,PC,9861,10415,pd0wx9w1,goiwbf,2017-10-06,2017-10-21T23:44:59Z,3,10
9,Super Mario Sunshine,Any%,2002-07-19,GameCube,7107,10249,v1pxjz68,sms,2002-07-19,2014-12-07T12:50:20Z,11,14


In [23]:
# Acquire icon and logo for each game
for id in top_50_games_data['ID']:
    get_game_cover(id)
    print(f"Downloaded cover and icon for game with id: {id}")

{'logo': {'uri': 'https://www.speedrun.com/static/theme/e87d4p8q/logo?v=b0fced9'}, 'cover-tiny': {'uri': 'https://www.speedrun.com/static/game/o1y9wo6q/cover?v=82fa0a4'}, 'cover-small': {'uri': 'https://www.speedrun.com/static/game/o1y9wo6q/cover?v=82fa0a4'}, 'cover-medium': {'uri': 'https://www.speedrun.com/static/game/o1y9wo6q/cover?v=82fa0a4'}, 'cover-large': {'uri': 'https://www.speedrun.com/static/game/o1y9wo6q/cover?v=82fa0a4'}, 'icon': {'uri': 'https://www.speedrun.com/static/theme/e87d4p8q/favicon?v=efb01e9'}, 'trophy-1st': {'uri': 'https://www.speedrun.com/static/theme/e87d4p8q/1st?v=efb01e9'}, 'trophy-2nd': {'uri': 'https://www.speedrun.com/static/theme/e87d4p8q/2nd?v=e792dc2'}, 'trophy-3rd': {'uri': 'https://www.speedrun.com/static/theme/e87d4p8q/3rd?v=de2a568'}, 'trophy-4th': {'uri': None}, 'background': {'uri': 'https://www.speedrun.com/static/theme/e87d4p8q/background?v=dc04a3b'}, 'foreground': {'uri': None}}
DOWNLOADING ICON
Downloaded cover and icon for game with id: o1